In [2]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime

# Function to parse multiple date formats
def parse_date(date_str):
    for fmt in ('%d-%m-%Y %H:%M', '%d/%m/%Y %H:%M'):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

# Load data from file
data = pd.read_csv('solar_madeupdata.csv')
# Convert 'datetime' column to datetime type with multiple formats handling
data['datetime'] = data['datetime'].apply(parse_date)

# Convert 'datetime' to numerical features (day, month, hour, minute)
data['day'] = data['datetime'].dt.day
data['month'] = data['datetime'].dt.month
data['hour'] = data['datetime'].dt.hour
data['minute'] = data['datetime'].dt.minute

# Drop original datetime column as it's not in a numerical format for the model
data = data.drop(columns=['datetime'])

# Handle categorical columns with OneHotEncoding
categorical_columns = ['conditions']  # Add any other string-based columns here
encoder = OneHotEncoder(sparse_output=False)
encoded_categories = encoder.fit_transform(data[categorical_columns])

# Convert encoded categories to DataFrame and concatenate with original data
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(categorical_columns))
data = pd.concat([data, encoded_df], axis=1)

# Drop the original categorical columns after encoding
data = data.drop(columns=categorical_columns)

# Separate features and target for solar output prediction
X = data.drop(columns=['temp', 'humidity', 'windspeed', 'cloudcover', 'solar_radiation', 'solar_energy', 'uv_index', 'ac_power_output_kW'])
y = data[['ac_power_output_kW']]  # Target is now just the AC power output

# Function to train model
def train_model(X_train, y_train):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    return model

# Initial training with first 1000 points
X_train = X.iloc[:1000]
y_train = y.iloc[:1000]
model = train_model(X_train, y_train)

# Start the loop for predictions and incremental training
index = 1000
while index + 24 <= len(X):
    # Predict next 24 data points for solar output
    X_next = X.iloc[index:index+24]
    y_pred = model.predict(X_next)
    
    # (Optional) Save predictions
    print("Predictions for next 24 points:", y_pred)
    
    # Simulate waiting for 10 seconds
    time.sleep(1)
    
    # Add the real 24 points from the data to the training set
    X_new = X.iloc[index:index+24]
    y_new = y.iloc[index:index+24]
    X_train = pd.concat([X_train, X_new])
    y_train = pd.concat([y_train, y_new])
    
    # Retrain the model
    model = train_model(X_train, y_train)
    
    # Move to the next chunk
    index += 24

# Load the predicted data from another file to predict solar output
predicted_data = pd.read_csv('predicted_data.csv')  # Replace with your actual file name
predicted_data['datetime'] = predicted_data['datetime'].apply(parse_date)  # Ensure date parsing
predicted_data['day'] = predicted_data['datetime'].dt.day
predicted_data['month'] = predicted_data['datetime'].dt.month
predicted_data['hour'] = predicted_data['datetime'].dt.hour
predicted_data['minute'] = predicted_data['datetime'].dt.minute
predicted_data = predicted_data.drop(columns=['datetime'])

# Handle categorical columns with OneHotEncoding for predicted data
encoded_predicted_categories = encoder.transform(predicted_data[categorical_columns])
encoded_predicted_df = pd.DataFrame(encoded_predicted_categories, columns=encoder.get_feature_names_out(categorical_columns))
predicted_data = pd.concat([predicted_data, encoded_predicted_df], axis=1)
predicted_data = predicted_data.drop(columns=categorical_columns)

# Make predictions on the new predicted data
solar_output_predictions = model.predict(predicted_data)

# (Optional) Save the predictions to a CSV file
predicted_data['solar_output_kW'] = solar_output_predictions
predicted_data.to_csv('Solar_Output_Predictions.csv', index=False)


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Predictions for next 24 points: [0.02519479 0.05422432 0.41033275 0.8114952  1.06280557 1.10744105
 0.63938102 0.29616457 0.52646129 0.35856188 0.155392   0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Predictions for next 24 points: [0.03626329 0.15466197 0.20493263 0.41234162 0.50541378 0.64890035
 0.79327276 0.94987039 0.60595794 0.24770245 0.06811963 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

Predictions for next 24 points: [0.04464135 0.46140351 1.28063114 2.15729737 3.04112393 3.60609443
 3.02701019 2.32717885 1.46729342 0.49913626 0.00785161 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Predictions for next 24 points: [0.03636215 0.36883528 1.18572125 2.17740088 3.02736355 3.33739035
 3.05181622 2.32529974 1.42175955 0.49565597 0.00435636 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'predicted_data.csv'